In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score
%load_ext autoreload
%autoreload 2

In [2]:
anho_actual = 2019
# Anho base, solo para calculos de diferencia y no usar dates
anho_inicio = 1987
campanha_actual = (anho_actual - anho_inicio)*18 + 7
campanha_actual

583

# Consultora

In [3]:
Consultora = pd.read_csv('datasets/maestro_consultora.csv').iloc[:, 1:]

In [4]:
Consultora.head()

,campanaingreso,campanaultimopedido,IdConsultora,estadocivil,flagsupervisor,campanaprimerpedido,flagconsultoradigital,flagcorreovalidad,edad,flagcelularvalidado
0,200907,201915,431,Casado(a),0,200907.0,0,0.0,58,1
1,199708,201914,2010,Casado(a),0,199708.0,0,NaN,58,1
2,199709,201914,3513,Casado(a),0,199709.0,0,NaN,60,1
3,201813,201911,4895,Casado(a),0,201813.0,0,NaN,50,1
4,199711,201914,6233,Casado(a),0,199711.0,0,NaN,65,1


In [5]:
# Edad medido en campanhas
campanha_ingreso = (Consultora['campanaingreso']//100 - anho_inicio)*18 + Consultora['campanaingreso']%100
# campanha_primer_pedido = (Consultora['campanaprimerpedido']//100 - anho_inicio)*18 + Consultora['campanaprimerpedido']%100
antiguedad = campanha_actual - campanha_ingreso
# n_campanha_vida
Consultora['antiguedad'] = antiguedad

In [6]:
edad, edad_labels = pd.qcut(Consultora['edad'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'], retbins=True)
# edad
Consultora['edad_cuartil'] = edad

In [7]:
consultora_filtered = pd.get_dummies(Consultora, columns=['edad_cuartil'])

consultora_filtered.head()

,campanaingreso,campanaultimopedido,IdConsultora,estadocivil,flagsupervisor,campanaprimerpedido,flagconsultoradigital,flagcorreovalidad,edad,flagcelularvalidado,antiguedad,edad_cuartil_Q1,edad_cuartil_Q2,edad_cuartil_Q3,edad_cuartil_Q4
0,200907,201915,431,Casado(a),0,200907.0,0,0.0,58,1,180,0,0,0,1
1,199708,201914,2010,Casado(a),0,199708.0,0,NaN,58,1,395,0,0,0,1
2,199709,201914,3513,Casado(a),0,199709.0,0,NaN,60,1,394,0,0,0,1
3,201813,201911,4895,Casado(a),0,201813.0,0,NaN,50,1,12,0,0,0,1
4,199711,201914,6233,Casado(a),0,199711.0,0,NaN,65,1,392,0,0,0,1


In [35]:
# This is interesting, some consultoras have asked for pedidos before actually be in the system
# (campanaingreso is greater than the first campanha in campanha_consultora database)
# just gonna leave it ther :v
# a = consultora_filtered.sort_values(by=['antiguedad'])
# a.head(20)

,campanaingreso,campanaultimopedido,IdConsultora,estadocivil,flagsupervisor,campanaprimerpedido,flagconsultoradigital,flagcorreovalidad,edad,flagcelularvalidado,antiguedad,edad_cuartil_Q1,edad_cuartil_Q2,edad_cuartil_Q3,edad_cuartil_Q4
62438,201915,201915,736513,Soltero(a),0,201915.0,0,0.0,20,1,-8,1,0,0,0
62779,201915,201915,730940,Otros,0,201915.0,0,1.0,48,1,-8,0,0,0,1
61683,201915,201915,685613,Soltero(a),0,201915.0,0,1.0,21,1,-8,1,0,0,0
91138,201915,201915,776144,Soltero(a),0,201915.0,0,0.0,24,1,-8,1,0,0,0
38656,201915,201915,745226,Soltero(a),0,201915.0,0,0.0,31,1,-8,0,1,0,0
68797,201915,201915,739616,Soltero(a),0,201915.0,0,0.0,20,0,-8,1,0,0,0
85395,201915,201915,764607,Soltero(a),0,201915.0,0,1.0,22,1,-8,1,0,0,0
88141,201915,201915,765879,Soltero(a),0,201915.0,0,0.0,19,1,-8,1,0,0,0
43503,201915,201915,647486,Soltero(a),0,201915.0,0,1.0,22,1,-8,1,0,0,0
45034,201915,201915,700747,Soltero(a),0,201915.0,0,0.0,29,1,-8,0,1,0,0


# Campanha

In [8]:
campanha_consultora = pd.read_csv('datasets/campana_consultora.csv').iloc[:,1:]
campanha_consultora = campanha_consultora.sort_values(by=['IdConsultora', 'campana'])
"""
    Droping: 
    - codigofactura
    - evaluacion_nuevas: should care about this
    - segmentacion: Consultara level
    - geografia
"""
campanha_consultora = campanha_consultora.select_dtypes(exclude=['object'])
campanha_consultora = campanha_consultora.drop(['codigocanalorigen'], axis=1)
# campanha_consultora.dtypes

In [9]:
campanha_consultora.head(10)
# campanha_consultora.dtypes

,campana,IdConsultora,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,flagpasopedidoweb,cantidadlogueos,flagdispositivo,flagdigital,flagofertadigital,flagsuscripcion
232,201807,125,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0.0,0
172361,201808,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0.0,0
320095,201809,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0.0,0
314240,201810,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0.0,0
526321,201811,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0.0,0
654608,201812,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0.0,0.0,0
7450,201807,202,1,1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0,0.0,0.0,0
137267,201808,202,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,0.0,0
358698,201809,202,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,0.0,0
296340,201810,202,1,1,1.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,1,0.0,0.0,0


# Detalle campanha

In [10]:
detalle_campanha = pd.read_csv('datasets/dtt_fvta_cl.csv')
detalle_campanha.rename(columns={'idconsultora': 'IdConsultora'}, inplace=True)
detalle_campanha.head()

/Users/raiszo/neu_env/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,campana,IdConsultora,codigotipooferta,descuento,ahorro,canalingresoproducto,idproducto,codigopalancapersonalizacion,palancapersonalizacion,preciocatalogo,...,realanulmnneto,realdevmnneto,realuuanuladas,realuudevueltas,realuufaltantes,realuuvendidas,realvtamnfaltneto,realvtamnneto,realvtamncatalogo,realvtamnfaltcatalogo
0,201807,544706,1,0.0,7990.0,NaN,200091158,12.0,NaN,5495.0,...,0.0,0.0,0,0,0,2,0.0,9236.0,10990.0,0.0
1,201807,638877,1,0.0,3800.0,WEB,200091762,1111.0,NaN,7690.0,...,0.0,0.0,0,0,0,1,0.0,6462.0,7690.0,0.0
2,201807,186016,1,0.0,3800.0,APP,200091762,42.0,NaN,7690.0,...,0.0,0.0,0,0,0,1,0.0,6462.0,7690.0,0.0
3,201807,579940,1,0.0,32748.0,WEB,200091762,12.0,NaN,6032.0,...,0.0,0.0,0,0,0,6,0.0,30414.0,36192.0,0.0
4,201807,342158,1,0.0,17272.0,WEB,200091762,12.0,NaN,7172.0,...,0.0,0.0,0,0,0,4,0.0,24108.0,28688.0,0.0


# Aggregations

In [43]:
campanha_agg = campanha_consultora.merge(
    consultora_filtered[['IdConsultora', 'campanaingreso', 'antiguedad', 'edad_cuartil_Q1', 'edad_cuartil_Q2', 'edad_cuartil_Q3', 'edad_cuartil_Q4']],
    on='IdConsultora', how='left')

In [44]:
campanha_agg.head()
# campanha_agg.dtypes

,campana,IdConsultora,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,...,flagdispositivo,flagdigital,flagofertadigital,flagsuscripcion,campanaingreso,antiguedad,edad_cuartil_Q1,edad_cuartil_Q2,edad_cuartil_Q3,edad_cuartil_Q4
0,201807,125,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,NaN,0.0,0,199707.0,396.0,0.0,0.0,0.0,1.0
1,201808,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,NaN,0.0,0,199707.0,396.0,0.0,0.0,0.0,1.0
2,201809,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,NaN,0.0,0,199707.0,396.0,0.0,0.0,0.0,1.0
3,201810,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,NaN,0.0,0,199707.0,396.0,0.0,0.0,0.0,1.0
4,201811,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,NaN,0.0,0,199707.0,396.0,0.0,0.0,0.0,1.0


In [45]:
a = campanha_agg.sort_values(by=['antiguedad'])
a[['IdConsultora', 'campana', 'antiguedad', 'campanaingreso']].head(20)

,IdConsultora,campana,antiguedad,campanaingreso
952094,739616,201810,-8.0,201915.0
1284048,776144,201814,-8.0,201915.0
1284049,776144,201815,-8.0,201915.0
1284050,776144,201816,-8.0,201915.0
1284051,776144,201817,-8.0,201915.0
987843,745226,201807,-8.0,201915.0
987844,745226,201808,-8.0,201915.0
987845,745226,201809,-8.0,201915.0
987846,745226,201810,-8.0,201915.0
410306,521732,201813,-8.0,201915.0


### Get sum and mean aggregations

In [13]:
# SUM
detalle_sum = detalle_campanha.groupby(
    ['IdConsultora', 'campana']
)['ahorro', 'realvtamncatalogo', 'realvtamnneto', 'realanulmnneto', 'realdevmnneto', 'realuuanuladas',
  'realuudevueltas', 'realuufaltantes', 'realuuvendidas', 'realvtamnfaltneto'].sum()
detalle_sum = pd.DataFrame(detalle_sum).reset_index()

detalle_sum.columns = ['IdConsultora', 'campana', 'ahorro_sum', 'realvtamncatalogo_sum',
       'realvtamnneto_sum', 'realanulmnneto_sum', 'realdevmnneto_sum', 'realuuanuladas_sum',
       'realuudevueltas_sum', 'realuufaltantes_sum', 'realuuvendidas_sum', 'realvtamnfaltneto_sum']
# detalle_sum.columns

# MEAN
detalle_mean = detalle_campanha.groupby(
    ['IdConsultora', 'campana']
)['ahorro', 'realvtamncatalogo', 'realvtamnneto', 'realanulmnneto', 'realdevmnneto', 'realuuanuladas',
  'realuudevueltas', 'realuufaltantes', 'realuuvendidas', 'realvtamnfaltneto'].sum()
detalle_mean = pd.DataFrame(detalle_mean).reset_index()

detalle_mean.columns = ['IdConsultora', 'campana', 'ahorro_mean', 'realvtamncatalogo_mean',
       'realvtamnneto_mean', 'realanulmnneto_mean', 'realdevmnneto_mean', 'realuuanuladas_mean',
       'realuudevueltas_mean', 'realuufaltantes_mean', 'realuuvendidas_mean', 'realvtamnfaltneto_mean']
# detalle_mean.columns

### Merge sum and mean

In [14]:
detalle_agg = detalle_sum.merge(detalle_mean, on=['IdConsultora', 'campana'])
detalle_agg.head()

,IdConsultora,campana,ahorro_sum,realvtamncatalogo_sum,realvtamnneto_sum,realanulmnneto_sum,realdevmnneto_sum,realuuanuladas_sum,realuudevueltas_sum,realuufaltantes_sum,...,ahorro_mean,realvtamncatalogo_mean,realvtamnneto_mean,realanulmnneto_mean,realdevmnneto_mean,realuuanuladas_mean,realuudevueltas_mean,realuufaltantes_mean,realuuvendidas_mean,realvtamnfaltneto_mean
0,202,201807,16379.0,35710.0,25022.0,0.0,0.0,0,0,0,...,16379.0,35710.0,25022.0,0.0,0.0,0,0,0,13,0.0
1,202,201810,19302.0,57680.0,39439.0,0.0,0.0,0,0,0,...,19302.0,57680.0,39439.0,0.0,0.0,0,0,0,17,0.0
2,202,201812,21338.0,52930.0,38142.0,0.0,0.0,0,0,0,...,21338.0,52930.0,38142.0,0.0,0.0,0,0,0,7,0.0
3,202,201813,8522.0,45810.0,29975.0,0.0,0.0,0,0,0,...,8522.0,45810.0,29975.0,0.0,0.0,0,0,0,10,0.0
4,202,201814,14378.0,51910.0,37335.0,0.0,0.0,0,0,0,...,14378.0,51910.0,37335.0,0.0,0.0,0,0,0,12,0.0


## Final merge

In [15]:
total_merge = campanha_agg.merge(detalle_agg, on=['IdConsultora', 'campana'], how='left')
total_merge = total_merge.sort_values(by=['IdConsultora', 'campana'])
# Remember that if Flagpasopedido is 0, no detail exist for that campaign, thus creating NaN columns :''v
total_merge.fillna(0, inplace=True)

In [16]:
total_merge.head()

,campana,IdConsultora,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,...,ahorro_mean,realvtamncatalogo_mean,realvtamnneto_mean,realanulmnneto_mean,realdevmnneto_mean,realuuanuladas_mean,realuudevueltas_mean,realuufaltantes_mean,realuuvendidas_mean,realvtamnfaltneto_mean
0,201807,125,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,201808,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,201809,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,201810,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,201811,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Check if Consultora asked for another pedido the next campanha
total_merge['flag_shift'] = total_merge.groupby('IdConsultora')['Flagpasopedido'].shift(-1)

In [18]:
total_merge[['IdConsultora', 'campana', 'flag_shift']].head(12)

,IdConsultora,campana,flag_shift
0,125,201807,0.0
1,125,201808,0.0
2,125,201809,0.0
3,125,201810,0.0
4,125,201811,0.0
5,125,201812,NaN
6,202,201807,0.0
7,202,201808,0.0
8,202,201809,1.0
9,202,201810,0.0


In [19]:
df_val = total_merge[total_merge['flag_shift'].isnull()].set_index(['IdConsultora', 'campana'])
df_model = total_merge[total_merge['flag_shift'].notna()].set_index(['IdConsultora', 'campana'])

In [46]:
df_model[['flag_shift', 'antiguedad']].head(12)

flag_shift  antiguedad
IdConsultora campana                        
125          201807          0.0       396.0
             201808          0.0       396.0
             201809          0.0       396.0
             201810          0.0       396.0
             201811          0.0       396.0
202          201807          0.0       396.0
             201808          0.0       396.0
             201809          1.0       396.0
             201810          0.0       396.0
             201811          1.0       396.0
             201812          1.0       396.0
             201813          1.0       396.0

In [21]:
X = df_model.drop(['flag_shift'], axis=1)
y = df_model['flag_shift']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
X_train.shape

(1102573, 39)

In [28]:
a = X_train['antiguedad']
a.where(a < 0)

IdConsultora  campana
73438         201807    NaN
304627        201816    NaN
436026        201903    NaN
749502        201809    NaN
793460        201901    NaN
                         ..
244908        201818    NaN
381254        201904    NaN
267023        201813    NaN
683107        201816    NaN
256834        201817    NaN
Name: antiguedad, Length: 1102573, dtype: float64

In [49]:
X_train['antiguedad']

IdConsultora  campana
73438         201807     414.0
304627        201816      19.0
436026        201903     120.0
749502        201809      19.0
793460        201901       8.0
                         ...  
244908        201818     222.0
381254        201904     139.0
267023        201813     199.0
683107        201816      44.0
256834        201817      30.0
Name: antiguedad, Length: 1102573, dtype: float64